# Version 15

In [ ]:
!pip install selenium
!pip install PyMySQL
!install SQLAlchemy


In [13]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.action_chains import ActionChains
import pymysql
import sqlalchemy
import pymysql
import numpy as np
pymysql.install_as_MySQLdb()
driver=webdriver.Chrome('/Users/oscarwan/Downloads/chromedriver')
url = 'https://albert.nyu.edu/albert_index.html'
driver.get(url)

/var/folders/bt/709h7c513ld30l7ynqg_cv0c0000gn/T/ipykernel_4464/2287114899.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('/Users/oscarwan/Downloads/chromedriver')


# Login to Alber. After finish loading, then run the following block

In [14]:
def reEnterCE(driver = driver):
    CEbutton = driver.find_elements(By.CSS_SELECTOR, '#nyuSSSHomeLinksStatic > div > div > ul > li:nth-child(4) > a')
    CEbutton[0].click()
    time.sleep(3)

    driver.switch_to.frame(2)
reEnterCE()

# Find all terms and schools names

In [16]:
selects = driver.find_elements(By.TAG_NAME, 'select')
terms = [selects[0].find_elements(By.TAG_NAME, 'option')[x+1].accessible_name for x in range(19)]
select = Select(selects[0])
select.select_by_visible_text(terms[0])
time.sleep(1)
schoolSelect = driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_DER_ACAD_ORG')
schools = [x.accessible_name for x in schoolSelect.find_elements(By.TAG_NAME, 'option')][1:]
temp = schools[1]
schools[1] = schools[5]
schools[5] = temp

# Initialize course evaluation dataframes and login to DB

In [19]:


courseEvaluationColumns =  ['courseCode',
 'instructor',
 'totalResponses',
 'responseRate',
 'OverallEvaluationCourse.',
 'objectivesClearlyStated',
 'courseWellOrganized',
 'intellectuallyStimulating',
 'EffectiveLearning',
 'informative',
 'challenging',
 'increasedKnowledge',
 'overallEvaluationInstructor.',
 'conduciveToLearning.',
 'helpfulFeedback',
 'helpedMeLearn.',
 'eencouragedParticipation.',
 'facilitateDiscussion',
 'openToQuestions',
 'accessibleToStudents',
 'inclusion']

courseEvaluationSternColumns = ['courseCode', 'instructor', 'totalResponses',
       'responseRate', 'OverallEvaluationCourse',
       'clearlyStatedObjectives',
       'wellOrganized',
       'intellectuallyStimulating',
       'overallEvaluationInstructor',
       'conduciveToLearning',
       'providedHelpfulFeedback']

courseEvaluation = pd.DataFrame(columns = courseEvaluationColumns)
courseEvaluationStern = pd.DataFrame(columns = courseEvaluationSternColumns)


In [17]:

conn_string = 'mysql://{user}:{password}@{host}:{port}/{db}?charset={encoding}'.format(
    user='daily_screamer', 
    password='sF5pxbMLBg8=', 
    host = 'jsedocc7.scrc.nyu.edu', 
    port     = 3306, 
    encoding = 'utf8',
    db = 'daily_screamer'
)
engine = sqlalchemy.create_engine(conn_string)

# Uility Functions and Scrapers

In [20]:
def keepTryingToClick(element, n = 5):
    for ii in range(n):
        error = 0
        try:
            element.click()
            time.sleep(.1)
        except Exception as e:
            print(e)
            error += 1
            pass

        if error > 0:
            print("trying to click again")
            #time.sleep(3)
        else:
            break

def keeyTryingToFind(elementCSS, n = 5):
    for ii in range(n):
        
        error = 0
        try:
            driver.find_element(By.CSS_SELECTOR, elementCSS)
            time.sleep(.1)
        except Exception as e:
            print(e)
            error += 1
            pass

        if error > 0:
            print("trying to find again")
            time.sleep(3)
        else:
            return driver.find_element(By.CSS_SELECTOR, elementCSS)

def waitForLoad(inputCSS):
    #print("waiting to load")
    Wait = WebDriverWait(driver, 90)
    Wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, inputCSS)))


lastScrapedSubjectIndex = 0
lastScrapedCourseIndex = 0


def scrapeSternCourse(driver = driver, courseEvaluationStern = courseEvaluationStern):
    questions = driver.find_elements(By.TAG_NAME, 'tr')
    courseCode = driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_WRK_DESCRLONG').text
    instructor_ = driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_CLS_VW2_NYU_INSTR_EVAL_Y').text
    totalResponses = driver.find_element(By.ID, 'NYU_CEV_PUB_WRK_NYU_CRSEVL_Q_TOT_R').text
    responseRate = driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_WRK_NYU_CRSEVL_PERCENT').text
    generalInfo = [courseCode, instructor_, responseRate, totalResponses]
    ratings = [x.find_elements(By.CLASS_NAME, 'ps_box-value')[1].text for x in questions[:-1]]
    courseEvaluationStern.loc[len(courseEvaluationStern.index)] = generalInfo + ratings


def scrapeCourse(driver = driver, courseEvaluation = courseEvaluation):
    questions = driver.find_elements(By.TAG_NAME, 'tr')
    courseCode = driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_WRK_DESCRLONG').text
    instructor_ = driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_CLS_VW2_NYU_INSTR_EVAL_Y').text
    totalResponses = driver.find_element(By.ID, 'NYU_CEV_PUB_WRK_NYU_CRSEVL_Q_TOT_R').text
    responseRate = driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_WRK_NYU_CRSEVL_PERCENT').text
    generalInfo = [courseCode, instructor_, responseRate, totalResponses]
    ratings = [x.find_elements(By.CLASS_NAME, 'ps_box-value')[1].text for x in questions[:-1]]
    courseEvaluation.loc[len(courseEvaluation.index)] = generalInfo + ratings

def scrapeOneSubject(driver = driver, isStern = False, courseContinueFrom = 0, courseEvaluation = courseEvaluation):
    global lastScrapedCourseIndex
    courses = driver.find_elements(By.TAG_NAME, 'tr')
    Wait = WebDriverWait(driver, 90)
    lastScrapedCourseIndex = courseContinueFrom
    for ii in range(courseContinueFrom, len(courses)-1):
        course = driver.find_elements(By.TAG_NAME, 'tr')[ii]
        if 'Global' in course.find_elements(By.CLASS_NAME, 'ps_box-value')[2].text:
            print(course.find_elements(By.CLASS_NAME, 'ps_box-value')[2].text)
            continue
        Wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'ps-button')))
        course.find_element(By.CLASS_NAME, 'ps-button').click()
        waitForLoad('#DESCR100_Q\$0')
        if driver.find_elements(By.CSS_SELECTOR, '#NYU_CEV_CLS_VW2_NYU_INSTR_EVAL_Y')[0].text.strip() == '':
            lastScrapedCourseIndex += 1
            # go back to class list
            keepTryingToClick(driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_WRK_RETURN_PB'))
            #driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_WRK_RETURN_PB').click()
            Wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'ps-button')))
            time.sleep(1)
            continue


        # click on "course eval"
        keepTryingToClick(driver.find_element(By.CSS_SELECTOR, '#DESCR100_Q\$0'))
        time.sleep(.1)

        # click on "instructor eval"
        try:
            keepTryingToClick(driver.find_element(By.CSS_SELECTOR, '#DESCR100_Q\$1'))
            time.sleep(.1)
        except:
            keepTryingToClick(driver.find_element(By.CSS_SELECTOR, '#DESCR100_Q\$2'))
            time.sleep(.1)

        if isStern:
            try:
                scrapeSternCourse(driver)
            except Exception as e:
                #print(e)
                print(driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_WRK_DESCRLONG').text, "not stored")
                pass
        else:
            try:
                scrapeCourse(driver, courseEvaluation=courseEvaluation)
            except Exception as e:
                #print(e)
                print(driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_WRK_DESCRLONG').text, "not stored")
                pass
        lastScrapedCourseIndex += 1
        # go back to class list
        keepTryingToClick(driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_WRK_RETURN_PB'))
        #driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_WRK_RETURN_PB').click()
        Wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'ps-button')))
        time.sleep(1)
        

def scrapeAllSubjects(driver= driver, isStern = False, continueFrom = 1, courseContinueFrom = 0, courseEvaluation = courseEvaluation):
    # find all subjects in that school
    global lastScrapedSubjectIndex
    lastScrapedSubjectIndex = continueFrom
    waitForLoad('#NYU_CEV_PUB_DER_DESCRFORMAL')
    subjectSelector = keeyTryingToFind('#NYU_CEV_PUB_DER_DESCRFORMAL')
    subjects = [x.accessible_name for x in subjectSelector.find_elements(By.TAG_NAME, 'option')][continueFrom:]
    for ii in subjects:
        subjectSelector = keeyTryingToFind('#NYU_CEV_PUB_DER_DESCRFORMAL')
        #driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_DER_DESCRFORMAL')
        #subjects = [x.accessible_name for x in subjectSelector.find_elements(By.TAG_NAME, 'option')][1:]
        selectSubject =Select(subjectSelector)
        selectSubject.select_by_visible_text(ii)
        time.sleep(1)

        #Wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#NYU_CEV_PUB_DER_SEARCH')))

        # click "Search Published Course Evaluation Results"
        keepTryingToClick(driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_DER_SEARCH'))
        # wait for the page to load
        #Wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr')))
        time.sleep(1)
        scrapeOneSubject(driver, isStern, courseContinueFrom, courseEvaluation=courseEvaluation)
        time.sleep(.1)

        # click "return to school... selection"
        keepTryingToClick(driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_WRK_RETURN_PB'))
        lastScrapedSubjectIndex += 1
        courseContinueFrom = 0

def scrapeAterm(term, driver = driver, continueFrom = 1, isStern = True, courseContinueFrom = 0, courseEvaluation=courseEvaluation):
    print("Scraping Term %s" %term)
    # Select term
    selects = driver.find_elements(By.TAG_NAME, 'select')
    select = Select(selects[0])
    select.select_by_visible_text(term)
    time.sleep(.1)
    # Select Stern
    schoolSelect = driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_DER_ACAD_ORG')
    selectSchool = Select(schoolSelect)
    if isStern:
        selectSchool.select_by_visible_text(schools[1])
    else:
        selectSchool.select_by_visible_text(schools[0])

    time.sleep(1)
    # scrape stern
    scrapeAllSubjects(isStern=isStern, driver = driver, continueFrom=continueFrom, courseContinueFrom = courseContinueFrom, courseEvaluation=courseEvaluation)
    print("finished scraping {term}".format(term = term))


def continueScrapingTermAfterStop(termIndex, driver = driver, isStern = False, continueFrom = 1, courseContinueFrom = 0, courseEvaluation=courseEvaluation):
    tempSubject = continueFrom
    tempCourse = courseContinueFrom
    ct = 0

    while ct <= 7:
        findWhereLeftOff()
        try:
            scrapeAterm(terms[termIndex], driver = driver, isStern = isStern, continueFrom= tempSubject, courseContinueFrom = tempCourse, courseEvaluation=courseEvaluation)
            engine = sqlalchemy.create_engine(conn_string)
            if not isStern:
                courseEvaluation.to_sql(name = 'CASCE', con = engine, if_exists = 'append', index = False)
            else:
                courseEvaluationStern.to_sql(name = 'SternCE', con = engine, if_exists = 'append', index = False)
        except Exception as e:
            print(e)
            ct += 1
            tempCourse = lastScrapedCourseIndex
            tempSubject = lastScrapedSubjectIndex
            engine = sqlalchemy.create_engine(conn_string)
            if not isStern:
                courseEvaluation.to_sql(name = 'CASCE', con = engine, if_exists = 'append', index = False)
            else:
                courseEvaluationStern.to_sql(name = 'SternCE', con = engine, if_exists = 'append', index = False)
            print("failed ", ct)
        else:
            break
            
def findWhereLeftOff():
    print("Subject Index: ", lastScrapedSubjectIndex)
    print("Course Index: ", lastScrapedCourseIndex)
    a = lastScrapedSubjectIndex
    b = lastScrapedCourseIndex
    return {'continueFrom':a, 'courseContinueFrom':b}
#findWhereLeftOff()

def uploadToDB(CE):
    CE.to_sql(name = 'CASCE', con = engine, if_exists = 'append', index = False)

def dropDBduplicates():
    temp = pd.read_sql("SELECT * FROM CASCE", engine).drop_duplicates()
    temp.to_sql(name = 'CASCE', con = engine, if_exists = 'replace', index = False)

def seeIfCourseExists(courseCode):
    return pd.read_sql("SELECT * FROM CASCE WHERE courseCode = '%s'" % courseCode, engine)

def checkIfCourseExistsInOneTerm(term, df, driver = driver, isStern = False, numCoursesToCheck = 3, numSubjectsToCheck = 3):
    Wait = WebDriverWait(driver, 90)
    numMiss = 0
    selects = driver.find_elements(By.TAG_NAME, 'select')
    select = Select(selects[0])
    select.select_by_visible_text(term)
    time.sleep(.1)
    # Select Stern
    schoolSelect = driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_DER_ACAD_ORG')
    selectSchool = Select(schoolSelect)
    schoolSelect = driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_DER_ACAD_ORG')
    selectSchool = Select(schoolSelect)
    if isStern:
        selectSchool.select_by_visible_text(schools[1])
    else:
        selectSchool.select_by_visible_text(schools[0])

    # Select 3 random subjects to check
    waitForLoad('#NYU_CEV_PUB_DER_DESCRFORMAL')
    subjectSelector = keeyTryingToFind('#NYU_CEV_PUB_DER_DESCRFORMAL')
    subjects = [x.accessible_name for x in subjectSelector.find_elements(By.TAG_NAME, 'option')][1:]
    subjectIndex = np.random.choice(range(1, len(subjects)), numSubjectsToCheck)
    time.sleep(.1)
    for ii in subjectIndex:
        subjectSelector = keeyTryingToFind('#NYU_CEV_PUB_DER_DESCRFORMAL')
        #driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_DER_DESCRFORMAL')
        #subjects = [x.accessible_name for x in subjectSelector.find_elements(By.TAG_NAME, 'option')][1:]
        selectSubject =Select(subjectSelector)
        selectSubject.select_by_visible_text(subjects[ii])
        time.sleep(.3)

        #Wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#NYU_CEV_PUB_DER_SEARCH')))

        # click "Search Published Course Evaluation Results"
        keepTryingToClick(driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_DER_SEARCH'))
        Wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'ps-button')))
        time.sleep(1)
        if len(driver.find_elements(By.CSS_SELECTOR, '#\#ICOK')) != 0:
            keepTryingToClick(driver.find_element(By.CSS_SELECTOR, '#\#ICOK'))
            keepTryingToClick(driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_WRK_RETURN_PB'))
            waitForLoad('#NYU_CEV_PUB_DER_DESCRFORMAL')
            continue
        # choose 2 random courses
        courses = driver.find_elements(By.TAG_NAME, 'tr')
        chooseCourseIndex = np.random.choice(range(len(courses)-1), numCoursesToCheck)

        for ii in chooseCourseIndex:
            course = driver.find_elements(By.TAG_NAME, 'tr')[ii]
            if 'Global' in course.find_elements(By.CLASS_NAME, 'ps_box-value')[2].text:
                print(course.find_elements(By.CLASS_NAME, 'ps_box-value')[2].text)
                continue
            Wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'ps-button')))
            course.find_element(By.CLASS_NAME, 'ps-button').click()
            waitForLoad('#DESCR100_Q\$0')
            courseCode = driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_WRK_DESCRLONG').text
            if courseCode not in df['courseCode'].values:
                print(courseCode, ' Not Stored. Storing Now')
                
                try:
                    keepTryingToClick(driver.find_element(By.CSS_SELECTOR, '#DESCR100_Q\$1'))
                    time.sleep(.1)
                except:
                    keepTryingToClick(driver.find_element(By.CSS_SELECTOR, '#DESCR100_Q\$2'))
                    time.sleep(.1)

                if isStern:
                    try:
                        scrapeSternCourse(driver, courseEvaluationStern)
                    except Exception as e:
                        #print(e)
                        print(driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_WRK_DESCRLONG').text, "not stored")
                        pass
                else:
                    try:
                        scrapeCourse(driver, courseEvaluation=courseEvaluation)
                    except Exception as e:
                        #print(e)
                        print(driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_WRK_DESCRLONG').text, "not stored")
                        pass
                numMiss += 1

            # go back to class list
            keepTryingToClick(driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_WRK_RETURN_PB')) #NYU_CEV_PUB_WRK_RETURN_PB
            #driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_WRK_RETURN_PB').click()
            Wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'ps-button')))

        keepTryingToClick(driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_WRK_RETURN_PB'))
        waitForLoad('#NYU_CEV_PUB_DER_DESCRFORMAL')
    return numMiss
    

# Scrape CAS 
replace the ... with the termIndex you're assgined to and run `continueScrapingTermAfterStop(termIndex=..., driver = driver)`

Wendy: 0, 1, 2, 3

Jason: 4, 5, 6, 7

Angela: 8, 9, 10, 11

Sylvia: 12, 13, 14, 15

Oscar: 16, 17, 18

In [13]:
terms
notStored = [8,9,10,12]
notStored = [terms[i] for i in notStored]
notStored

['09 - Summer 2020', '10 - January 2020', '11 - Fall 2019', '13 - Spring 2019']

In [ ]:
termIndex = 12 # replace with the index of the term you're scraping
#continueScrapingTermAfterStop(termIndex=termIndex, driver = driver)

# if the scraping unexpectedly quit/stopped, uncomment the following line and comment the above line, and then run
continueScrapingTermAfterStop(termIndex=termIndex, driver = driver, courseEvaluation=courseEvaluation, **findWhereLeftOff())

In [93]:
for term in terms[5:]:
    print("chekcing,", term)
    checkIfCourseExistsInOneTerm(term, pd.read_sql('SELECT * FROM SternCE', engine), driver, numCoursesToCheck=5, numSubjectsToCheck=5, isStern=True)

chekcing, 06 - Spring 2021
NYU Shanghai (Global)
NYU London (Global)
chekcing, 07 - January 2021
SOIM-UB 6 040  (1218) - Law, Business & Society  (Lecture)  Not Stored. Storing Now
SOIM-UB 6 040  (1218) - Law, Business & Society  (Lecture) not stored
SOIM-UB 6 41  (1524) - Law, Business & Society  (Lecture)  Not Stored. Storing Now
SOIM-UB 6 41  (1524) - Law, Business & Society  (Lecture) not stored
SOIM-UB 6 41  (1524) - Law, Business & Society  (Lecture)  Not Stored. Storing Now
SOIM-UB 6 41  (1524) - Law, Business & Society  (Lecture) not stored
SOIM-UB 6 41  (1524) - Law, Business & Society  (Lecture)  Not Stored. Storing Now
SOIM-UB 6 41  (1524) - Law, Business & Society  (Lecture) not stored
SOIM-UB 6 040  (1218) - Law, Business & Society  (Lecture)  Not Stored. Storing Now
SOIM-UB 6 040  (1218) - Law, Business & Society  (Lecture) not stored
MKTG-UB 40 040  (1448) - Entertainment & Media Industries  (Lecture)  Not Stored. Storing Now
MKTG-UB 40 040  (1448) - Entertainment & Medi

NoSuchElementException: Message: Could not locate element with visible text: Leonard N. Stern School of Business - UGRD


In [89]:
questions = driver.find_elements(By.TAG_NAME, 'tr')
courseCode = driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_WRK_DESCRLONG').text
instructor_ = driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_CLS_VW2_NYU_INSTR_EVAL_Y').text
totalResponses = driver.find_element(By.ID, 'NYU_CEV_PUB_WRK_NYU_CRSEVL_Q_TOT_R').text
responseRate = driver.find_element(By.CSS_SELECTOR, '#NYU_CEV_PUB_WRK_NYU_CRSEVL_PERCENT').text
generalInfo = [courseCode, instructor_, responseRate, totalResponses]
ratings = [x.find_elements(By.CLASS_NAME, 'ps_box-value')[1].text for x in questions[:-1]]
courseEvaluationStern.loc[len(courseEvaluationStern.index)] = generalInfo + ratings

In [80]:
pd.read_sql('SELECT * FROM SternCE', engine).drop_duplicates()

,courseCode,instructor,totalResponses,responseRate,OverallEvaluationCourse,clearlyStatedObjectives,wellOrganized,intellectuallyStimulating,overallEvaluationInstructor,conduciveToLearning,providedHelpfulFeedback
0,ACCT-UB 1 001 (3103) - Prin of Financial Acct...,"Yiwei Dou, Xiaojing Meng",34.4%,11,4.3,4.5,4.6,4.4,4.6,4.5,4.5
1,FINC-UB 86 040 (1222) - Chinese Financial Mar...,Matthew Richardson,39.0%,16,4.8,4.8,4.4,4.8,4.7,4.7,4.5
2,MGMT-UB 1 040 (1221) - Management and Organiz...,Elinor Flynn,33.3%,9,4.8,4.8,4.9,4.8,4.8,4.9,4.6
3,MGMT-UB 1 041 (1224) - Management and Organiz...,Katia Meggiorin,32.1%,9,4.3,4.4,4.6,4.4,4.7,4.8,3.9
4,FINC-UB 86 040 (1114) - Chinese Financial Mar...,Matthew Richardson,57.7%,15,4.4,4.4,4.4,4.2,4.7,4.5,3.8
...,...,...,...,...,...,...,...,...,...,...,...
9189,STAT-UB 103 006 (10954) - Stats F/Bus Cntl Re...,Halina Frydman,56.9%,41,3.5,4.1,3.7,3.5,3.3,3.5,3.8
9190,MKTG-UB 1 040 (1194) - Intro to Marketing (L...,Carol Pluzinski,15.7%,8,4.6,5.0,4.8,4.5,4.9,4.8,4.5
9191,MKTG-UB 40 040 (1195) - Entertainment & Media...,Kenneth J Krushel,93.3%,14,4.7,4.6,4.8,4.4,4.9,4.8,4.5
9329,MKTG-UB 40 002 (11515) - Entertainment & Medi...,Kenneth J Krushel,56.1%,23,4.7,4.7,4.7,4.6,4.7,4.5,4.6


In [79]:
dropDBduplicates()

,courseCode,instructor,totalResponses,responseRate,OverallEvaluationCourse.,objectivesClearlyStated,courseWellOrganized,intellectuallyStimulating,EffectiveLearning,informative,...,increasedKnowledge,overallEvaluationInstructor.,conduciveToLearning.,helpfulFeedback,helpedMeLearn.,eencouragedParticipation.,facilitateDiscussion,openToQuestions,accessibleToStudents,inclusion
0,"ANTH-UA 1 001 (2752) - Culture, Power, Societ...",Murray Laura,27.8%,5,4.8,4.4,4.6,4.4,4.2,4.4,...,4.4,4.6,4.4,4.4,4.4,4.6,4.2,4.6,4.4,4.4
1,"ANTH-UA 1 001 (2722) - Culture, Power, Societ...",Anisha Chadha,30.0%,6,4.7,4.8,4.8,4.8,4.7,4.5,...,4.7,4.8,4.7,4.7,4.7,5.0,4.8,4.8,4.8,4.8
2,"ANTH-UA 1 060 (2798) - Culture, Power, Societ...",Anisha Chadha,30.0%,6,4.7,4.8,4.8,4.8,4.7,4.5,...,4.7,4.8,4.7,4.7,4.7,5.0,4.8,4.8,4.8,4.8
3,ANTH-UA 2 001 (2732) - Human Evolution (Lect...,Alejandra Ortiz,28.6%,4,3.8,4.0,4.3,4.0,3.5,4.5,...,4.3,4.3,4.0,4.5,4.3,3.3,2.8,3.5,4.3,4.3
4,ANTH-UA 2 001 (2723) - Human Evolution (Lect...,paola cerrito,17.1%,6,3.7,3.7,4.0,4.0,3.8,4.0,...,3.8,3.7,3.7,3.0,3.0,3.8,3.0,3.8,4.0,3.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9653,SOC-UA 936 001 (25624) - Adv Sem: (Seminar),Tyson Patros,92.9%,13,5.0,5.0,4.8,5.0,5.0,5.0,...,5.0,4.9,5.0,4.9,5.0,4.9,4.8,5.0,5.0,5.0
9654,SOC-UA 941 001 (10614) - Seminar: (Seminar),"Paul Heideman, Ruth Horowitz",42.9%,3,4.7,4.7,4.7,4.0,4.3,4.3,...,4.7,4.7,4.7,5.0,4.7,5.0,4.7,4.7,5.0,5.0
9655,SOC-UA 950 001 (9004) - Senior Honors Researc...,"Paul Heideman, Robert Jackson",46.2%,6,2.5,2.7,2.3,2.2,2.2,1.7,...,2.2,2.8,2.8,2.2,2.3,4.5,3.2,4.0,4.2,3.5
9656,SPAN-UA 2 001 (8918) - Spanish for Beginners ...,NADIA BERENICE VILLAFUERTE VAZQUEZ,23.1%,3,4.7,4.3,4.3,4.7,4.3,4.3,...,4.7,4.7,4.7,4.3,4.7,4.7,4.3,4.7,4.7,4.7


# Uploading data, and Check if data is uploaded
1. to see whether a specific course is uploaded, run `seeIfCourseExists(courseCode=...)`. Replace with the courseCode.

2. to see whether a term is uploaded, run `checkIfCourseExistsInOneTerm(term = ..., df = pd.read_sql('SELECT * FROM CASCE', engine))` replace ... with the **NAME** of the term

3. If you find a term not being uploaded, try `uploadToDB(courseEvaluation)` first, and then check 2.. If still not uploaded, scrape again.

In [ ]:
df = pd.read_sql("SELECT * FROM CASCE", engine)

a = ... # the first index you're assigned to
b = ... # the last index you're assigned to +1

termsNotStored = []
for term in terms[a:b]:
    if checkIfCourseExistsInOneTerm(term, temp) > 3:
        termsNotStored.append(term)

termsNotStored 

In [6]:
c = engine.execute('show tables')
print(c.fetchall())

[('CASCE',), ('SternCE',), ('stern_course_evaluations',)]


In [21]:
pd.read_sql("SELECT * FROM stern_course_evaluations", engine)

,Academic Year,Term,Q2. Overall Eval of Course,Subject/Number/Section,Contact
0,AY 2019 - 2020,Spring 2020,4.5,ACCT-UB 33.001,Paul Zarowin
1,AY 2019 - 2020,Spring 2020,4.3,ACCT-GB 6302.C1,Paul Zarowin
2,AY 2019 - 2020,Spring 2020,4.8,ACCT-GB 3310.30,Paul Zarowin
3,AY 2019 - 2020,Spring 2020,4.6,ACCT-GB 3310.M1,Paul Zarowin
4,AY 2019 - 2020,Fall 2019,4.4,ACCT-GB 2302.M1,Paul Zarowin
...,...,...,...,...,...
1707,AY 2019 - 2020,Summer 2020,4.2,MKTG-UB 1.002,Fanglin Chen
1708,AY 2019 - 2020,Summer 2020,3.8,ACCT-UB 1.003,Danye Wang
1709,AY 2019 - 2020,Summer 2020,4.1,STAT-UB 1.002,Zhuoyi Yang
1710,AY 2019 - 2020,Winter 2020,4.8,MGMT-UB 1.041,Raphael Martins


In [28]:
pd.read_csv('https://github.com/thunder-applications/technical-project-deadline-12-09-22-oscarwzt/blob/6cfe368c67566b7354c5c0c0124beec7d53b36d4/backend/raw_data/playbyplay.csv')

HTTPError: HTTP Error 404: Not Found